# SQL for Data Analyst
- [Basic][1]
- [Join][2]
- [Aggregation][3]
- [Subqueries][3]
[1]:http://127.0.0.1:8888/notebooks/SQL4DA/Basic.ipynb
[2]:http://127.0.0.1:8888/notebooks/SQL4DA/Join.ipynb
[3]:http://127.0.0.1:8888/notebooks/SQL4DA/Aggregation.ipynb
[4]:http://127.0.0.1:8888/notebooks/SQL4DA/Subqueries.ipynb

# SQL Subqueries & Temporary Tables
Both subqueries and table expressions are methods for being able to write a query that creates a table, and then write a query that interacts with this newly created table. Sometimes the question you are trying to answer doesn’t have an answer when working directly with existing tables in database.  
* [Subqueries][1]
    - [Exercise][2]
* [WITH CTE][3]
    - [Exercise][4] 
[1]:#Subqueries
[2]:#Exercise
[3]:#WITH-(-Common-Table-Expression，CTE-)
[4]:#Exercise-2

In [1]:
%load_ext autoreload
%autoreload 2
from Query import *
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
database = 'test'
exercise = 'subquery.txt'

In [3]:
Query = query(database)
#Query.get_table()
#Query.sql2db('parch-and-posey.sql')
Query.connect()

## Subqueries
Subqueries Whenever we need to use existing tables to `create a new table that we then want to query again`, this is an indication that we will need to use some sort of subquery.  
#### Example
```SQL
  SELECT channel, AVG(event_count)
  FROM 
  (SELECT
    DATE_TRUNC('day', occurred_at) AS day,
    COUNT(\*) AS event_count,
    channel
  FROM web_events
  GROUP BY 1,3)sub
  GROUP BY 2
```
#### Well Formatted Query
```SQL
SELECT *
  FROM (SELECT DATE_TRUNC('day',occurred_at) AS day,
                  channel, COUNT(*) AS events
        FROM web_events
        GROUP BY 1,2
        ORDER BY 3 DESC) sub;
```

In [41]:
com = readcom(exercise)
result = Query.execute(com)
result

SELECT channel, AVG(total)
FROM
(SELECT date(w.occurred_at) AS Date, w.channel, COUNT(*) total
FROM web_events w
GROUP BY 1, 2)sub
GROUP BY 1
ORDER BY 2 DESC


,channel,AVG(total)
0,direct,4.896488
1,organic,1.667250
2,facebook,1.598347
3,adwords,1.570191
4,twitter,1.316667
5,banner,1.289973


In [48]:
com = readcom(exercise)
result = Query.execute(com)
result

SELECT 
    strftime('%Y%m',o.occurred_at) AS Time, 
    AVG(o.standard_qty),
    AVG(o.gloss_qty),
    AVG(o.poster_qty)
FROM orders o
WHERE strftime('%Y%m',o.occurred_at) = 
(SELECT strftime('%Y%m',MIN(o.occurred_at)) min_month
FROM orders o)
GROUP BY 1


,Time,AVG(o.standard_qty),AVG(o.gloss_qty),AVG(o.poster_qty)
0,201312,268.222222,208.949495,111.818182


## Exercise
### 1. Provide the name of the sales_rep in each region with the largest amount of total_amt_usd sales.
```SQL
SELECT 
    sale_rep, region, MAX(sales)
    FROM
    (SELECT s.name sale_rep, r.name region, MAX(o.total_amt_usd) sales
        FROM orders o
        JOIN accounts a ON a.id = o.account_id
        JOIN sales_reps s ON s.id = a.sales_rep_id
        JOIN region r ON r.id = s.region_id
        GROUP BY 1, 2)sub
    GROUP BY 2
    ORDER BY 1;
```

In [53]:
com = readcom(exercise)
result = Query.execute(com)
result

SELECT 
    sale_rep, region, MAX(sales)
    FROM
    (SELECT s.name sale_rep, r.name region, MAX(o.total_amt_usd) sales
        FROM orders o
        JOIN accounts a ON a.id = o.account_id
        JOIN sales_reps s ON s.id = a.sales_rep_id
        JOIN region r ON r.id = s.region_id
        GROUP BY 1, 2)sub
    GROUP BY 2
    ORDER BY 1


,sale_rep,region,MAX(sales)
0,Cliff Meints,Midwest,48675.90
1,Dawna Agnew,West,232207.07
2,Dorotha Seawell,Southeast,95005.82
3,Necole Victory,Northeast,93106.81


### 2. For the region with the largest (sum) of sales total_amt_usd, how many total (count) orders were placed?
```SQL
SELECT 
    r.name region, COUNT(*) total_orders
    FROM orders o
    JOIN accounts a ON a.id = o.account_id
    JOIN sales_reps s ON s.id = a.sales_rep_id
    JOIN region r ON r.id = s.region_id
    GROUP BY 1
    HAVING SUM(o.total_amt_usd) =
        (SELECT MAX(total_sales)
            FROM
            (SELECT r.name, SUM(o.total_amt_usd) total_sales
                FROM orders o
                JOIN accounts a ON a.id = o.account_id
                JOIN sales_reps s ON s.id = a.sales_rep_id
                JOIN region r ON r.id = s.region_id
                GROUP BY 1
            )sub
        );
```

In [65]:
com = readcom(exercise)
result = Query.execute(com)
result

SELECT 
    r.name region, COUNT(*) total_orders
    FROM orders o
    JOIN accounts a ON a.id = o.account_id
    JOIN sales_reps s ON s.id = a.sales_rep_id
    JOIN region r ON r.id = s.region_id
    GROUP BY 1
    HAVING SUM(o.total_amt_usd) =
        (SELECT MAX(total_sales)
            FROM
            (SELECT r.name, SUM(o.total_amt_usd) total_sales
                FROM orders o
                JOIN accounts a ON a.id = o.account_id
                JOIN sales_reps s ON s.id = a.sales_rep_id
                JOIN region r ON r.id = s.region_id
                GROUP BY 1
            )sub
        )


,region,total_orders
0,Northeast,2357


### 3. How many accounts had more total purchases than the account name which has bought the most standard_qty paper throughout their lifetime as a customer?
```SQL
SELECT a.name
    FROM accounts a
    JOIN orders o ON o.account_id = a.id
    HAVING SUM(o.total) > 
            (SELECT SUM(o.total) total_purchases
                FROM accounts a
                JOIN orders o ON o.account_id = a.id
                WHERE a.name =
                    (SELECT name, MAX(total_std)
                        FROM
                        (SELECT a.name, SUM(o.standard_qty) total_std
                            FROM accounts a
                            JOIN orders o ON o.account_id = a.id
                            GROUP BY 1
                        )sub
                        GROUP BY 1
                    )
            )
```

```SQL
SELECT COUNT(*)
    FROM
    (SELECT a.name, SUM(o.total)
        FROM accounts a
        JOIN orders o ON o.account_id = a.id
        GROUP BY 1
        HAVING SUM(o.total) > 
                            (SELECT total
                                FROM
                                (SELECT a.name acc_name, SUM(o.standard_qty) total_std, SUM(o.total) total
                                    FROM accounts a
                                    JOIN orders o ON o.account_id = a.id
                                    GROUP BY 1
                                    ORDER BY 2 DESC
                                    LIMIT 1
                                )sub
                            )
    )sub;
```

In [87]:
com = readcom(exercise)
result = Query.execute(com)
result

SELECT COUNT(*)
    FROM
    (SELECT a.name, SUM(o.total)
        FROM accounts a
        JOIN orders o ON o.account_id = a.id
        GROUP BY 1
        HAVING SUM(o.total) > 
                            (SELECT total
                                FROM
                                (SELECT a.name acc_name, SUM(o.standard_qty) total_std, SUM(o.total) total
                                    FROM accounts a
                                    JOIN orders o ON o.account_id = a.id
                                    GROUP BY 1
                                    ORDER BY 2 DESC
                                    LIMIT 1
                                )sub
                            )
    )sub


,COUNT(*)
0,3


### 4. For the customer that spent the most (in total over their lifetime as a customer) total_amt_usd, how many web_events did they have for each channel?
```SQL
SELECT w.channel, COUNT(*) events
    FROM web_events w
    JOIN accounts a ON a.id = w.account_id
    WHERE a.name =
    (SELECT acc_name
        FROM
        (SELECT a.name acc_name, SUM(o.total_amt_usd) total_sales
            FROM accounts a
            JOIN orders o ON o.account_id = a.id
            GROUP BY 1
            ORDER BY 2 DESC
            LIMIT 1
        )sub
    )
    GROUP BY 1
    ORDER BY events DESC;
```

In [93]:
com = readcom(exercise)
result = Query.execute(com)
result

SELECT w.channel, COUNT(*) events
    FROM web_events w
    JOIN accounts a ON a.id = w.account_id
    WHERE a.name =
    (SELECT acc_name
        FROM
        (SELECT a.name acc_name, SUM(o.total_amt_usd) total_sales
            FROM accounts a
            JOIN orders o ON o.account_id = a.id
            GROUP BY 1
            ORDER BY 2 DESC
            LIMIT 1
        )sub
    )
    GROUP BY 1
    ORDER BY events DESC


,channel,events
0,direct,44
1,organic,13
2,adwords,12
3,facebook,11
4,twitter,5
5,banner,4


### 5. What is the lifetime average amount spent in terms of total_amt_usd for the top 10 total spending accounts?
```SQL
SELECT AVG(total_sales) avg_of_top10
    FROM
    (SELECT a.name acc_name, SUM(o.total_amt_usd) total_sales
        FROM accounts a
        JOIN orders o ON o.account_id = a.id
        GROUP BY 1
        ORDER BY 2 DESC
        LIMIT 10)top_ten;
```

In [95]:
com = readcom(exercise)
result = Query.execute(com)
result

SELECT AVG(total_sales) avg_of_top10
 FROM
 (SELECT a.name acc_name, SUM(o.total_amt_usd) total_sales
     FROM accounts a
     JOIN orders o ON o.account_id = a.id
     GROUP BY 1
     ORDER BY 2 DESC
     LIMIT 10)top_ten


,avg_of_top10
0,304846.969


### 6. What is the lifetime average amount spent in terms of total_amt_usd, including only the companies that spent more per order, on average, than the average of all orders.
```SQL
SELECT AVG(company_avg)
    FROM
    (SELECT o.account_id target, AVG(o.total_amt_usd) company_avg
        FROM orders o
        GROUP BY 1
        HAVING AVG(o.total_amt_usd) >
        (SELECT AVG(o.total_amt_usd) avg_sales
             FROM orders o)
    );
```

In [111]:
com = readcom(exercise)
result = Query.execute(com)
result

SELECT AVG(company_avg)
    FROM
    (SELECT o.account_id target, AVG(o.total_amt_usd) company_avg
        FROM orders o
        GROUP BY 1
        HAVING AVG(o.total_amt_usd) >
        (SELECT AVG(o.total_amt_usd) avg_sales
             FROM orders o)
    )


,AVG(company_avg)
0,4721.139744


# WITH ( Common Table Expression，CTE )

The `WITH` statement is often called a Common Table Expression or CTE. Though these expressions serve the exact <mark>same purpose as subqueries</mark>, they are <mark>more common</mark> in practice, as they tend to be <mark>cleaner</mark> for a future reader to follow the logic.

The general sequence of steps to execute a WITH clause is:
1. Initiate the `WITH`
2. Specify the expression name for the to-be-defined query.
3. Optional: Specify column names separated by commas.
4. After assigning the name of the expression, enter the AS command. The expressions, in this case, are the named result sets that you will use later in the main query to refer to the CTE.
5. Write the query required to produce the desired temporary data set.
6. If working with more than one CTEs or `WITH` clauses, initiate each subsequent one separated by a comma and repeat steps 2-4. Such an arrangement is also called a nested WITH clause.
7. Reference the expressions defined above in a subsequent query using `SELECT`, `INSERT`, `UPDATE`, `DELETE`, or `MERGE`

#### Quick Example
QUESTION: You need to find the average number of events for each channel per day.
SOLUTION:
```SQL
--without with satement
SELECT channel, AVG(events) AS average_events
FROM (
    SELECT DATE_TRUNC('day',occurred_at) AS day, channel, COUNT(*) as events
    FROM web_events
    GROUP BY 1,2
    ) sub
GROUP BY channel
ORDER BY 2 DESC;

--rewrite with satement
WITH events AS (
              SELECT DATE_TRUNC('day',occurred_at) AS day, channel, COUNT(*) as events
              FROM web_events
              GROUP BY 1,2
              )

SELECT channel, AVG(events) AS average_events
FROM events
GROUP BY channel
ORDER BY 2 DESC;
```

## Exercise 2
### 1. Provide the name of the sales_rep in each region with the largest amount of total_amt_usd sales.
```SQL
WITH sum AS(
     SELECT s.name s_name, r.name region, SUM(o.total_amt_usd) sum
        FROM accounts a
        JOIN orders o ON o.account_id = a.id
        JOIN sales_reps s ON a.sales_rep_id = s.id
        JOIN region r ON s.region_id = r.id
        GROUP BY 1, 2
),
max AS(
    SELECT region, MAX(sum.sum) largest_sales
    FROM sum
    GROUP BY 1
)
SELECT sum.s_name, sum.region, sum.sum
FROM (sum)
JOIN (max)
ON max.largest_sales = sum.sum AND max.region = sum.region;
```

In [10]:
com = readcom(exercise)
result = Query.execute(com)
result

WITH sum AS(
     SELECT s.name s_name, r.name region, SUM(o.total_amt_usd) sum
        FROM accounts a
        JOIN orders o ON o.account_id = a.id
        JOIN sales_reps s ON a.sales_rep_id = s.id
        JOIN region r ON s.region_id = r.id
        GROUP BY 1, 2
),
max AS(
    SELECT region, MAX(sum.sum) largest_sales
    FROM sum
    GROUP BY 1
)
SELECT sum.s_name, sum.region, sum.sum
FROM (sum)
JOIN (max)
ON max.largest_sales = sum.sum AND max.region = sum.region


,s_name,region,sum
0,Charles Bidwell,Midwest,675637.19
1,Earlie Schleusner,Southeast,1098137.72
2,Georgianna Chisholm,West,886244.12
3,Tia Amato,Northeast,1010690.60


### 2. For the region with the largest sales total_amt_usd, how many total orders were placed?
```SQL
WITH total AS(
    SELECT r.name, SUM(o.total_amt_usd) total_sales
        FROM orders o
        JOIN accounts a ON a.id = o.account_id
        JOIN sales_reps s ON s.id = a.sales_rep_id
        JOIN region r ON r.id = s.region_id
        GROUP BY 1
),
    largest AS(
    SELECT MAX(total_sales)
        FROM total
)
SELECT 
    r.name region, COUNT(*) total_orders
    FROM orders o
    JOIN accounts a ON a.id = o.account_id
    JOIN sales_reps s ON s.id = a.sales_rep_id
    JOIN region r ON r.id = s.region_id
    GROUP BY 1
    HAVING SUM(o.total_amt_usd) = (SELECT * FROM largest);
```

In [6]:
com = readcom(exercise)
result = Query.execute(com)
result

WITH total AS(
    SELECT r.name, SUM(o.total_amt_usd) total_sales
        FROM orders o
        JOIN accounts a ON a.id = o.account_id
        JOIN sales_reps s ON s.id = a.sales_rep_id
        JOIN region r ON r.id = s.region_id
        GROUP BY 1
),
    largest AS(
    SELECT MAX(total_sales)
        FROM total
)
SELECT 
    r.name region, COUNT(*) total_orders
    FROM orders o
    JOIN accounts a ON a.id = o.account_id
    JOIN sales_reps s ON s.id = a.sales_rep_id
    JOIN region r ON r.id = s.region_id
    GROUP BY 1
    HAVING SUM(o.total_amt_usd) = (SELECT * FROM largest)


,region,total_orders
0,Northeast,2357


3. How many accounts had more total purchases than the account name which has bought the most standard_qty paper throughout their lifetime as a customer?
```SQL
WITH most AS(
    SELECT o.account_id id, SUM(o.standard_qty) total_std, SUM(o.total) total
    FROM orders o
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 1
),
num AS(
    SELECT a.id
    FROM accounts a
    JOIN orders o ON o.account_id = a.id
    GROUP BY 1
    HAVING SUM(o.total) > (SELECT total FROM most)
)
SELECT COUNT(*) FROM num;
```

In [23]:
com = readcom(exercise)
result = Query.execute(com)
result

WITH most AS(
    SELECT o.account_id id, SUM(o.standard_qty) total_std, SUM(o.total) total
    FROM orders o
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 1
),
num AS(
    SELECT a.id
    FROM accounts a
    JOIN orders o ON o.account_id = a.id
    GROUP BY 1
    HAVING SUM(o.total) > (SELECT total FROM most)
)
SELECT COUNT(*) FROM num


,COUNT(*)
0,3


4. For the customer that spent the most (in total over their lifetime as a customer) total_amt_usd, how many web_events did they have for each channel?
```SQL
WITH most AS(
    SELECT a.name name, SUM(o.total_amt_usd)
    FROM accounts a
    JOIN orders o ON o.account_id = a.id
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 1
)
SELECT w.channel, COUNT(*) events_num
    FROM web_events w
    JOIN accounts a ON a.id = w.account_id
    WHERE a.name = (SELECT most.name FROM most)
GROUP BY 1
ORDER BY 2 DESC;
```

In [25]:
com = readcom(exercise)
result = Query.execute(com)
result

WITH most AS(
    SELECT a.name name, SUM(o.total_amt_usd)
    FROM accounts a
    JOIN orders o ON o.account_id = a.id
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 1
)
SELECT w.channel, COUNT(*)
FROM web_events w
JOIN accounts a ON a.id = w.account_id
WHERE a.name = (SELECT most.name FROM most)
GROUP BY 1
ORDER BY 2 DESC


,channel,COUNT(*)
0,direct,44
1,organic,13
2,adwords,12
3,facebook,11
4,twitter,5
5,banner,4


5. What is the lifetime average amount spent in terms of total_amt_usd for the top 10 total spending accounts?

In [ ]:
com = readcom(exercise)
result = Query.execute(com)
result

6. What is the lifetime average amount spent in terms of total_amt_usd, including only the companies that spent more per order, on average, than the average of all orders.

In [ ]:
com = readcom(exercise)
result = Query.execute(com)
result